## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-27-01-11-00 +0000,wsj,Milei Wins Mandate for Free-Market Revolution ...,https://www.wsj.com/world/americas/milei-wins-...
1,2025-10-27-01-01-15 +0000,nyt,President Oversees Signing of Cambodia-Thailan...,https://www.nytimes.com/live/2025/10/25/us/tru...
2,2025-10-27-01-00-25 +0000,nyt,"Keeping the House Absent, Johnson Marginalizes...",https://www.nytimes.com/2025/10/25/us/politics...
3,2025-10-27-01-00-00 +0000,wsj,The U.S. Evacuated Them from Afghanistan—Then ...,https://www.wsj.com/politics/policy/afghan-ref...
4,2025-10-27-01-00-00 +0000,wsj,More Big Companies Bet They Can Still Grow Wit...,https://www.wsj.com/business/companies-hiring-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2323/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
74,trump,38
66,melissa,9
88,will,8
65,hurricane,8
100,mamdani,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
180,2025-10-26-06-49-24 +0000,nypost,"Trump signs peace, trade deals ending conflict...",https://nypost.com/2025/10/26/us-news/trump-si...,75
121,2025-10-26-13-39-19 +0000,bbc,Will Trump's 'peace deal' solve Thailand-Cambo...,https://www.bbc.com/news/videos/cy40n3ykx93o?a...,73
102,2025-10-26-16-00-33 +0000,bbc,US and China agree framework of trade deal ahe...,https://www.bbc.com/news/articles/c1lqj5lz4geo...,70
100,2025-10-26-16-10-00 +0000,wsj,Trump Says He Will Raise Tariffs on Canada by ...,https://www.wsj.com/politics/policy/trump-hike...,69
133,2025-10-26-12-24-18 +0000,nypost,"US, China reach deal to allow TikTok sale, Bes...",https://nypost.com/2025/10/26/us-news/us-china...,65


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
180,75,2025-10-26-06-49-24 +0000,nypost,"Trump signs peace, trade deals ending conflict...",https://nypost.com/2025/10/26/us-news/trump-si...
110,41,2025-10-26-15-13-13 +0000,nypost,Sen. Chris Murphy suggests Dems could end shut...,https://nypost.com/2025/10/26/us-news/sen-chri...
159,39,2025-10-26-09-47-45 +0000,nypost,2 Louvre robbery suspects busted in $100M muse...,https://nypost.com/2025/10/26/world-news/louvr...
14,37,2025-10-26-23-18-16 +0000,nypost,Zohran Mamdani insists NYPD Commish Jessica Ti...,https://nypost.com/2025/10/26/us-news/mamdani-...
29,31,2025-10-26-21-41-25 +0000,nypost,Sex-abuse vics slam NYC Council speaker for bl...,https://nypost.com/2025/10/26/us-news/sex-abus...
120,31,2025-10-26-13-48-30 +0000,nypost,California Gov. Gavin Newsom lays out timeline...,https://nypost.com/2025/10/26/us-news/gavin-ne...
179,30,2025-10-26-07-02-02 +0000,nypost,Hurricane Melissa strengthens into Category 3 ...,https://nypost.com/2025/10/26/world-news/hurri...
168,27,2025-10-26-09-00-00 +0000,wsj,Rep. Jared Golden didn’t just side with the GO...,https://www.wsj.com/politics/policy/jared-gold...
17,24,2025-10-26-22-42-20 +0000,nypost,LIRR employees in sprawling fake-ID scam made ...,https://nypost.com/2025/10/26/us-news/lirr-emp...
65,24,2025-10-26-19-22-10 +0000,nypost,Russia’s new ‘invincible’ nuclear missile is r...,https://nypost.com/2025/10/26/world-news/russi...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
